#Introdução à inteligência artificial

##Projeto final - Classificação de modelos de carros

Este arquivo deve ser entregue até dia __04/08/2020__. Para isso, adicione o seu RA no título do arquivo para identificação.
A finalidade deste projeto é modificar o **classificador** de uma rede neural previamente treinada, onde fica a seu critério escolher qual arquitetura do modelo VGG você irá utilizar. 
Será avaliado quais foram as camadas utilizadas, funções de ativação, métodos para evitar *overfitting*, critério de perda, otimizador e quaisquer outros métodos que alterem os hiperparâmetros da rede.

A precisão percentual será calculada pelo algoritmo no final deste arquivo.

O dataset a ser usado está disponível através do link: https://ai.stanford.edu/~jkrause/cars/car_dataset.html

### Importação das bibliotecas necessárias

In [ ]:
# Importação das bibliotecas
from __future__ import print_function, division

import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
import numpy as np
import torchvision
from torchvision import datasets, models, transforms
import matplotlib.pyplot as plt
import time
import os
import copy
from torch.utils.data.sampler import SubsetRandomSampler
from torch.utils.data import Dataset

### Download das imagens do dataset

In [ ]:
!wget http://imagenet.stanford.edu/internal/car196/car_ims.tgz

### Extração das imagens

In [ ]:
!tar -xvf car_ims.tgz

### Download das anotações do dataset

Aqui serão baixados as informações do dataset:

1. Nomes das classes
2. Definição de qual foto pertence a qual classe
3. Quais fotos serão usadas para treinamento, validação e teste.

In [ ]:
!wget http://imagenet.stanford.edu/internal/car196/cars_annos.mat

### Análise das anotações

In [ ]:
# Ler o arquivo com as anotações
from scipy.io import loadmat
cars_dataset_annotations = loadmat('cars_annos.mat')

In [ ]:
#Verificar o conteúdo das anotações
cars_dataset_annotations

### Separando as imagens

In [ ]:
# Verifica a posição do arquivo do projeto
root = os.getcwd()
# Cria as pastas para cada tipo de dado
train_path = root + "/train"
test_path = root + "/test"
# Verifica se existe para evitar erro
if not os.path.exists(train_path):
  os.mkdir(train_path)
if not os.path.exists(test_path):
  os.mkdir(test_path)
new_path = ""
move_path=""
# Analisa quais os nomes das classes para criar pastas
for class_name in cars_dataset_annotations['class_names'][0]:
  for data_type in ["train","test"]:
    # Caso alguma classe tiver o caracter "/" trocar por outro para não haver erro de caminho
    if "/" in class_name[0]:
     class_name[0] = class_name[0].replace("/","_")
    if data_type == "train":
      new_path = train_path + "/" + class_name[0]
    elif data_type == "test":
      new_path = test_path + "/" + class_name[0]
    if not os.path.exists(new_path):
      os.mkdir(new_path)
# Analisa qual o tipo da imagem e move ela para cada pasta
for data in cars_dataset_annotations['annotations'][0]:
  car_class = cars_dataset_annotations['class_names'][0][data[5][0][0]-1][0] 
  if "/" in car_class:
    car_class = car_class.replace("/","_")
  file_name = data[0][0].split("/")[1]
  if(data[6][0][0] == 0):
    move_path = train_path + "/" + car_class  + "/" + file_name
  if(data[6][0][0] == 1):
    move_path = test_path + "/" + car_class  + "/" + file_name
  os.rename(root+"/"+data[0][0],move_path)    

### Definição das transformações, tamanho de lote e carregamento das imagens

In [ ]:
# Número de subprocessos a serem usados para carregamento de dados
num_workers = 0

# Quantas amostras por lote 
batch_size = 8

# Porcentagem de treinamento definida para uso como validação
valid_size = 0.2

# Define as transformações, fique a vontade para adicionar mais transformações!!!
train_transform = transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.RandomVerticalFlip(),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])])
test_transform = transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])])

train_dataset = datasets.ImageFolder(train_path,train_transform)
test_dataset =  datasets.ImageFolder(test_path,test_transform)

# Obter índices de treinamento que serão usados para validação
num_train = len(train_dataset)
indices = list(range(num_train))
np.random.shuffle(indices)
split = int(np.floor(valid_size * num_train))
train_idx, valid_idx = indices[split:], indices[:split]

# Definir amostradores para obter lotes de treinamento e validação
train_sampler = SubsetRandomSampler(train_idx)
valid_sampler = SubsetRandomSampler(valid_idx)

# Preparar carregadores de dados (combinar conjunto de dados e amostrador)
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size,
    sampler=train_sampler, num_workers=num_workers)
valid_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, 
    sampler=valid_sampler, num_workers=num_workers)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size, 
    num_workers=num_workers)

class_names = train_dataset.classes
dataset_sizes = {'train':len(train_loader.dataset.samples)*(1-valid_size),'valid':len(valid_loader.dataset.samples)*valid_size,'test':len(test_loader.dataset.samples)}
data_loader = {'train':train_loader,'valid':valid_loader,'test':test_loader}

# Verifica qual hardware será utilizado
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [ ]:
# Função auxiliar
def imshow(inp, title=None):
    inp = inp.numpy().transpose((1, 2, 0))
    mean = np.array([0.485, 0.456, 0.406])
    std = np.array([0.229, 0.224, 0.225])
    inp = std * inp + mean
    inp = np.clip(inp, 0, 1)
    plt.figure(num=None, figsize=(25, 6), dpi=80, facecolor='w', edgecolor='k')
    plt.imshow(inp)
    if title is not None:
      plt.title(title)
    plt.pause(0.001)


# Pega um lote de treinamento
inputs, classes = next(iter(data_loader['train']))

# Faz um "grid" com as imagens
out = torchvision.utils.make_grid(inputs)

# Mostra um lote de imagens
imshow(out, title=[class_names[x] for x in classes])

In [ ]:
def train_model(model, criterion, optimizer, scheduler, num_epochs=25):
    since = time.time()

    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0

    for epoch in range(num_epochs):
        print('Época {}/{}'.format(epoch, num_epochs - 1))
        print('-' * 10)

        # Cada época tem uma fase de treino e validação
        for phase in ['train', 'valid']:
            if phase == 'train':
                model.train()  # Modelo em treinamento
            else:
                model.eval()   # Modelo em avaliação

            running_loss = 0.0
            running_corrects = 0

            for inputs, labels in data_loader[phase]:
                inputs = inputs.to(device)
                labels = labels.to(device)

                # Zera o gradiente do otimizador
                optimizer.zero_grad()

                # Analisa somente as perdas se for no treinamento
                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)
                    _, preds = torch.max(outputs, 1)
                    loss = criterion(outputs, labels)

                    # 'loss.backward()' + 'optimizer.step()' somente no treinamento
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                # Estatisticas
                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)
            if phase == 'train':
                scheduler.step()

            epoch_loss = running_loss / dataset_sizes[phase]
            epoch_acc = running_corrects.double() / dataset_sizes[phase]

            print('{} Perda: {:.4f} Precisão: {:.4f}'.format(
                phase, epoch_loss, epoch_acc))

            # Copia o modelo
            if phase == 'valid' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = copy.deepcopy(model.state_dict())

        print()

    time_elapsed = time.time() - since
    print('Treinamento completo em {:.0f}m {:.0f}s'.format(
        time_elapsed // 60, time_elapsed % 60))
    print('Melhor precisão: {:4f}'.format(best_acc))
    torch.save(best_model_wts,"model.pt")

    # Carrega os pesos do melhor modelo
    model.load_state_dict(best_model_wts)
    return model

### Modelo

Defina seu modelo utilizando o modelo [VGG](https://pytorch.org/docs/stable/torchvision/models.html) **pré-treinado**  e altere o classificador da rede neural.

In [ ]:
# TODO: Defina aqui qual VGG você utilizará, lembre-se que é aconselhavel usar a rede pré-treinada
model = NotImplemented

for param in model.parameters():
    param.requires_grad = False
   
# Quantidade de entradas do classificador antigo
num_ftrs = model.classifier[0].in_features

# TODO: Defina o novo classificador com entrada de num_ftrs e saída de len(class_names)
model.classifier = nn.Sequential(nn.Linear(num_ftrs,x),
                                 .
                                 .
                                 .

                                 nn.Linear(y,len(class_names)))

# Move o modelo para o dispositivo disponivel
model = model.to(device)

# TODO: Defina o critério
criterion = NotImplemented

# TODO: Defina o otimizador
optimizer = NotImplemented

# TODO: Defina o "scheduler"
exp_lr_scheduler = NotImplemented

### Treinamento

In [ ]:
model = train_model(model, criterion, optimizer, exp_lr_scheduler,
                       num_epochs=1)

### Carregar o modelo

In [ ]:
# Carrega o modelo 
file_name = "model.pt"
model.load_state_dict(torch.load(file_name))

### Avaliação do projeto

In [ ]:
# Faça o teste final para avaliação da rede
test_loss = 0.0
class_correct = list(0. for i in range(len(class_names)))
class_total = list(0. for i in range(len(class_names)))

train_on_gpu = torch.cuda.is_available()
model.eval()

for batch_idx, (data, target) in enumerate(data_loader['test']):
    # Move os tensores para a GPU se disponivel
    if train_on_gpu:
        data, target = data.cuda(), target.cuda()
    # Inferência 
    output = model(data)
    # Calcula a perda
    loss = criterion(output, target)
    # Atualiza a perda
    test_loss += loss.item()*data.size(0)
    # Convert as probabilidades para classe e escolhe somente a maior
    _, pred = torch.max(output, 1)    
    # Compara as predições com a classe verdadeira
    correct_tensor = pred.eq(target.data.view_as(pred))
    correct = np.squeeze(correct_tensor.numpy()) if not train_on_gpu else np.squeeze(correct_tensor.cpu().numpy())
    # Calcula a precisão para cada objeto
    for i in range(batch_size):
        if(correct.size==batch_size):
          label = target.data[i]
          class_correct[label] += correct[i].item()
          class_total[label] += 1

# Perda média
test_loss = test_loss/len(test_loader.dataset)
print('Perda de teste: {:.6f}\n'.format(test_loss))

for i in range(len(class_names)):
    if class_total[i] > 0:
      print('Precisão de teste de %5s: %2d%% (%2d/%2d)' % (
            class_names[i], 100 * class_correct[i] / class_total[i],
            np.sum(class_correct[i]), np.sum(class_total[i])))
    else:
        print('Precisão de teste de %5s: N/A (sem dados de treinamento)' % (class_names[i]))
overall_acc = 100. * np.sum(class_correct) / np.sum(class_total)
print('\nPrecisão de teste geral: %6.2f%% (%3d/%3d)' % (
    overall_acc,
    np.sum(class_correct), np.sum(class_total)))